In [1]:
!nvidia-smi

Wed Jul  5 12:44:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.9 MB/s eta 0:00:

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.8 MB/s eta 0:00:00
Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2
Found existing installation: accelerate 0.20.3
Uninstalling accelerate-0.20.3:
  Successfully uninstalled accelerate-0.20.3
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)


In [25]:
from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
device

'cuda'

In [11]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [13]:
!unzip summarizer-data.zip

Archive:  summarizer-data.zip
replace samsum-test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: samsum-test.csv         
replace samsum-train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: samsum-train.csv        
replace samsum-validation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: samsum-validation.csv   
replace samsum_dataset/dataset_dict.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: samsum_dataset/dataset_dict.json  
replace samsum_dataset/test/data-00000-of-00001.arrow? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: samsum_dataset/test/data-00000-of-00001.arrow  
replace samsum_dataset/test/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: samsum_dataset/test/dataset_info.json  
replace samsum_dataset/test/state.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: samsum_dataset/test/state.json  
replace samsum_dataset/train/data-00000-of-00001.arrow? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  infl

In [14]:
dataset_samsum = load_from_disk("samsum_dataset")
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [19]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]
print(f"Split lengths:{split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("***********************************************")
print(f"Dialogue: {dataset_samsum['test'][1]['dialogue']}")
print("************************************************")
print(f"summary:{dataset_samsum['test'][1]['summary']}")

Split lengths:[14732, 819, 818]
Features: ['id', 'dialogue', 'summary']
***********************************************
Dialogue: Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)
************************************************
summary:Eric and Rob are going to watch a stand-up on youtube.


In [20]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True)
  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch["summary"], max_length = 128, truncation = True)
  return {
      'input_ids':input_encodings['input_ids'],
      'attention_mask':input_encodings['attention_mask'],
      'labels':target_encodings['input_ids']
  }

In [21]:
datset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [22]:
datset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [24]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

In [26]:
trainer_args = TrainingArguments(
    output_dir = 'pegasus-samsum', num_train_epochs = 1, warmup_steps = 500, per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1, weight_decay = 0.01, logging_steps = 10, evaluation_strategy = 'steps', eval_steps = 500,
    save_steps = 1e6, gradient_accumulation_steps = 16
)

In [29]:
trainer = Trainer(model = model_pegasus, args = trainer_args, tokenizer = tokenizer, data_collator = seq2seq_data_collator,
                  train_dataset = datset_samsum_pt["test"], eval_dataset = datset_samsum_pt["validation"])

In [30]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0461518156762217, metrics={'train_runtime': 203.7522, 'train_samples_per_second': 4.02, 'train_steps_per_second': 0.25, 'total_flos': 312908679512064.0, 'train_loss': 3.0461518156762217, 'epoch': 1.0})

In [31]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

In [39]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size = 16, device = device, column_text = "article",
                                column_summary = "highlights"):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))
  for article_batch, target_batch in tqdm(zip(article_batches, target_batches)):
    inputs = tokenizer(article_batch, max_length = 1024, truncation = True, padding = "max_length", return_tensors = "pt")
    summaries = model.generate(input_ids = inputs["input_ids"].to(device),
                               attention_mask = inputs["attention_mask"].to(device),
                               length_penalty = 0.8, num_beams = 8, max_lenth = 128)
    decoded_summaries = [tokenizer.decode(s, skip_special_tokens = True, clean_up_tokenization_spaces = True) for s in summaries]
    decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
    metric.add_batch(predictions = decoded_summaries, references = target_batch)

  score = metric.comput()
  return score

In [40]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

In [42]:
score = calculate_metric_on_test_ds(datset_samsum_pt['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, device = device, column_text = 'dialogue', column_summary = 'summary')
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = [f'pegasus'])

0it [00:00, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│ in calculate_metric_on_test_ds:7                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1271 in generate        │
│                                                                                                  │
│   1268 │   │   generation_config = copy.deepcopy(generation_config)                              │
│   1269 │   │   model_kwargs = generation_config.update(**kwargs)  # All unused kwargs must be m  │
│   1270 │   │   generation_config.validate()                                                      │
│ ❱ 1271 │   │   self._validate_model_kwargs(model_kwargs.copy())                                  │
│   1272 │   │                                                                                     │
│   1273 │   │   # 2. Set generation parameters if not already defined                             │
│   1274 │   │   logits_processor = logits_processor if logits_processor is not None else LogitsP  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1144 in                 │
│ _validate_model_kwargs                                                                           │
│                                                                                                  │
│   1141 │   │   │   │   unused_model_args.append(key)                                             │
│   1142 │   │                                                                                     │
│   1143 │   │   if unused_model_args:                                                             │
│ ❱ 1144 │   │   │   raise ValueError(                                                             │
│   1145 │   │   │   │   f"The following `model_kwargs` are not used by the model: {unused_model_  │
│   1146 │   │   │   │   " generate arguments will also show up in this list)"                     │
│   1147 │   │   │   )                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: The following `model_kwargs` are not used by the model: ['max_lenth'] (note: typos in the generate 
arguments will also show up in this list)

In [43]:
model_pegasus.save_pretrained("pegasus-samsum-model")

In [44]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')